# Tokistar Materials Report

In [1]:
import pandas as pd
import numpy as np

### Read datafiles

In [2]:
inv = pd.read_csv('./data/data.txt')
bl = pd.read_csv('./data/bl.txt')
hfr = pd.read_csv('./data/hfr.txt')
validate = pd.read_csv('./data/validate.csv')
translate = pd.read_csv('./data/translate.csv')

### Set datafile dtypes

In [3]:
cols = inv.columns[1:]
inv[cols] = inv[cols].astype(float)
cols = bl.columns[1:]
bl[cols] = bl[cols].astype(float)
cols = hfr.columns[1:]
hfr[cols] = hfr[cols].astype(float)
cols = translate.columns[1:]
translate[cols] = translate[cols].astype(float)

### Set datafile column names

In [4]:
labels = ['Part Number', 'On Hand', 'On Order', 'Reorder']
inv.columns = labels
labels = ['Part Number', 'Backlog', 'Factor']
bl.columns = labels
labels = ['Part Number', 'HFR', 'Factor']
hfr.columns = labels
labels = ['Part Number', 'Valid PN']
validate.columns = labels
labels = ['Part Number', 'Factor']
translate.columns = labels

### Clean bl & hfr

In [5]:
def clean_sales_table(df):
    name = (list(df.columns))[1]
    rows = df.index
    df = df.loc[df['Part Number'] != " "]
    df = df.loc[df['Part Number'] != "LOT"]
    df = df.loc[df['Part Number'] != "MARK"]
    for row in df.index:
        df.loc[row, name] = df.loc[row, name] * df.loc[row,'Factor']
    df = df.drop('Factor', axis=1)
    df = df.groupby(['Part Number'], as_index=False).sum()
    return df


In [6]:
bl = clean_sales_table(bl)
hfr = clean_sales_table(hfr)

### Download schedule

In [7]:
tables = pd.read_html('http://www.toki.co.jp/purchasing/TLIHTML.files/sheet001.htm')

### Clean schedule

In [8]:
schedule = tables[0].fillna(0)
labels = ['Part Number'] + list(schedule.loc[3][5:])
schedule = schedule.drop([1,2,3,4], axis=1)
schedule = schedule[5:]
schedule.columns = labels
cols = schedule.columns[1:]
schedule[cols] = schedule[cols].astype(float)
schedule = schedule.groupby(['Part Number'], as_index=False).sum()

### Translate schedule

In [9]:
schedule = schedule.merge(translate, how='left').fillna(0)
rows = schedule[schedule['Factor'] != 0].index
cols = schedule.columns[1:]
for col in cols:
    schedule.loc[rows, col] = schedule.loc[rows,col] * schedule.loc[rows,'Factor']
schedule = schedule.drop('Factor', axis=1)

### Validate schedule

In [10]:
def fix_pn(toki, tli):
    if tli == 0:
        return toki
    if toki != tli:
        return tli
    else:
        return toki


In [11]:
schedule = validate.merge(schedule, how='right').fillna(0)
schedule['Part Number'] = schedule[['Part Number','Valid PN']].apply(lambda schedule: fix_pn(schedule['Part Number'], schedule['Valid PN']), axis=1)
schedule = schedule.drop('Valid PN', axis=1)

### Build materials

In [12]:
materials = pd.DataFrame()

In [13]:
materials[['Part Number', 'On Hand']] = inv[['Part Number', 'On Hand']]
materials = materials.merge(bl, how='left').fillna(0)
materials['Released'] = 0.0
materials = materials.merge(hfr, how='left').fillna(0)
materials['Released'] = materials['Backlog'] - materials['HFR']
materials['On Order'] = inv['On Order']
materials['T-Avail'] = materials['On Hand'] + materials['On Order'] - materials['Backlog']
materials['R-Avail'] = materials['T-Avail'] - materials['HFR']
materials['Reorder'] = inv['Reorder']
materials = materials.merge(schedule, how='left').fillna(0)
cols = materials.columns[1:]
materials[cols] = materials[cols].astype(int)

### Format and save files

In [14]:
df = materials
df_writer = pd.ExcelWriter("./data/materials.xlsx", engine="xlsxwriter")
df.to_excel(df_writer, index=False, sheet_name="Materials")
df_wb = df_writer.book
df_ws = df_writer.sheets["Materials"]

ro = df[(df['Reorder'] > df['T-Avail'])]
ro_writer = pd.ExcelWriter("./data/reorder.xlsx", engine="xlsxwriter")
ro.to_excel(ro_writer, index=False, sheet_name="Report")
ro_wb = ro_writer.book
ro_ws = ro_writer.sheets["Report"]

ro_hdr_pn = ro_wb.add_format({
    'bold': True,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'left',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

ro_hdr_body = ro_wb.add_format({
    'bold': True,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'right',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

ro_hdr_schedule = ro_wb.add_format({
    'bold': True,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'center',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

ro_pn = ro_wb.add_format({
    'bold': False,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'left',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

ro_body = ro_wb.add_format({
    'bold': False,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'right',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

ro_schedule = ro_wb.add_format({
    'bold': False,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'center',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

df_hdr_pn = df_wb.add_format({
    'bold': True,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'left',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

df_hdr_body = df_wb.add_format({
    'bold': True,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'right',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

df_hdr_schedule = df_wb.add_format({
    'bold': True,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'center',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

df_pn = df_wb.add_format({
    'bold': False,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'left',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

df_body = df_wb.add_format({
    'bold': False,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'right',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

df_schedule = df_wb.add_format({
    'bold': False,
    'text_wrap': False,
    'valign': 'bottom',
    'align': 'center',
    'fg_color': '#FFFFFF',
    'font_color': '#000000',
    'border': 1,
    'border_color': 'E0E0E0'
})

ro_ws.set_column("A:A", 20, ro_pn)
ro_ws.set_column("B:I", 10, ro_body)
ro_ws.set_column("J:Z", 10, ro_schedule)
df_ws.set_column("A:A", 20, df_pn)
df_ws.set_column("B:I", 10, df_body)
df_ws.set_column("J:Z", 10, df_schedule)

ro_ws.write(0, 0, ro.columns.values[0], ro_hdr_pn)
df_ws.write(0, 0, df.columns.values[0], df_hdr_pn)

for i in range(1, 9):
    ro_ws.write(0, i, ro.columns.values[i], ro_hdr_body)
    df_ws.write(0, i, df.columns.values[i], df_hdr_body)
for i in range(9, len(df.columns)):
    ro_ws.write(0, i, ro.columns.values[i], ro_hdr_schedule)
    df_ws.write(0, i, df.columns.values[i], df_hdr_schedule)

ro_writer.save()
df_writer.save()

ro = ro.set_index('Part Number')
df = df.set_index('Part Number')

In [15]:
ro

,On Hand,Backlog,Released,HFR,On Order,T-Avail,R-Avail,Reorder,2022/2/4,2022/3/4,2022/4/15
Part Number,,,,,,,,,,,
AV-CH-SA,0,0,0,0,0,0,0,500,0,0,0
AV-LCH-WH,480,0,0,0,0,480,480,500,0,0,0
AV-MC,1279,76,76,0,0,1203,1203,2500,0,0,0
AVLED-IW,4574,0,0,0,0,4574,4574,5000,0,0,0
BB-301,4440,0,0,0,0,4440,4440,5000,0,0,0
CABLE-BK,30498,3077,2066,1011,22000,49421,48410,50000,0,0,0
CL-203,7121,0,0,0,0,7121,7121,10000,0,0,0
CL-30,2794,0,0,0,0,2794,2794,3280,0,0,0
CR-B,4500,0,0,0,0,4500,4500,5000,0,0,0


In [16]:
df

,On Hand,Backlog,Released,HFR,On Order,T-Avail,R-Avail,Reorder,2022/2/4,2022/3/4,2022/4/15
Part Number,,,,,,,,,,,
AV-201,8544,0,0,0,0,8544,8544,1000,0,0,0
AV-BASE-BK,17800,0,0,0,0,17800,17800,5000,0,0,0
AV-BASE-SNAP,17120,0,0,0,0,17120,17120,10000,0,0,0
AV-BASE-WH,43355,380,380,0,0,42975,42975,25000,0,0,0
AV-BOX,126,0,0,0,0,126,126,100,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
XB-5F,12215,0,0,0,0,12215,12215,0,0,0,0
XB-5FB,1242,0,0,0,0,1242,1242,0,0,0,0
XB-8F,25130,0,0,0,0,25130,25130,0,0,0,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1206 entries, AV-201 to XB-8FU
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   On Hand    1206 non-null   int64
 1   Backlog    1206 non-null   int64
 2   Released   1206 non-null   int64
 3   HFR        1206 non-null   int64
 4   On Order   1206 non-null   int64
 5   T-Avail    1206 non-null   int64
 6   R-Avail    1206 non-null   int64
 7   Reorder    1206 non-null   int64
 8   2022/2/4   1206 non-null   int64
 9   2022/3/4   1206 non-null   int64
 10  2022/4/15  1206 non-null   int64
dtypes: int64(11)
memory usage: 113.1+ KB
